# voiage Validation Notebook

This notebook validates the core methods of the voiage library by comparing results with known examples and demonstrating the functionality.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from voiage.analysis import DecisionAnalysis
from voiage.schema import ParameterSet, ValueArray, TrialDesign, DecisionOption
from voiage.methods.sample_information import evsi
from voiage.plot.ceac import plot_ceac
from voiage.plot.voi_curves import plot_evpi_vs_wtp, plot_evsi_vs_sample_size

## Simple Health Economic Model

Let's create a simple health economic model with two treatment strategies: Standard Care and New Treatment.

In [ ]:
def simple_health_economic_model(parameter_samples):
    """Simple health economic model with two strategies.
    
    Args:
        parameter_samples: ParameterSet with parameters for the model
        
    Returns:
        ValueArray: Net benefit values for each strategy
    """
    # Extract parameters
    if isinstance(parameter_samples, ParameterSet):
        params = parameter_samples.parameters
    else:
        params = parameter_samples
    
    # Effectiveness parameters (QALYs)
    if 'effect_standard' in params:
        effect_standard = params['effect_standard']
    else:
        effect_standard = np.full_like(next(iter(params.values())), 0.7)
        
    if 'effect_new' in params:
        effect_new = params['effect_new']
    else:
        effect_new = np.full_like(next(iter(params.values())), 0.8)
    
    # Cost parameters (£)
    if 'cost_standard' in params:
        cost_standard = params['cost_standard']
    else:
        cost_standard = np.full_like(next(iter(params.values())), 2000)
        
    if 'cost_new' in params:
        cost_new = params['cost_new']
    else:
        cost_new = np.full_like(next(iter(params.values())), 3000)
    
    # Willingness-to-pay threshold (£ per QALY)
    wtp_thresholds = np.linspace(0, 100000, 20)
    
    # Calculate net benefits for each WTP threshold
    n_samples = len(effect_standard)
    n_strategies = 2
    n_wtp = len(wtp_thresholds)
    
    net_benefits = np.zeros((n_samples, n_strategies, n_wtp))
    
    for i, wtp in enumerate(wtp_thresholds):
        # Standard care net benefit
        net_benefits[:, 0, i] = effect_standard * wtp - cost_standard
        # New treatment net benefit
        net_benefits[:, 1, i] = effect_new * wtp - cost_new
    
    # Create ValueArray
    import xarray as xr
    dataset = xr.Dataset(
        {"net_benefit": (("n_samples", "n_strategies", "n_wtp"), net_benefits)},
        coords={
            "n_samples": np.arange(n_samples),
            "n_strategies": np.arange(n_strategies),
            "n_wtp": np.arange(n_wtp),
            "strategy": ("n_strategies", ["Standard Care", "New Treatment"])
        }
    )
    
    return ValueArray(dataset=dataset)

## Generate Parameter Samples

Let's generate parameter samples for our probabilistic sensitivity analysis (PSA).

In [ ]:
# Generate parameter samples
np.random.seed(42)  # For reproducibility
n_samples = 1000

# Effectiveness parameters (QALYs)
effect_standard = np.random.normal(0.7, 0.05, n_samples)  # Mean 0.7, SD 0.05
effect_new = np.random.normal(0.8, 0.05, n_samples)       # Mean 0.8, SD 0.05

# Cost parameters (£)
cost_standard = np.random.normal(2000, 200, n_samples)    # Mean £2000, SD £200
cost_new = np.random.normal(3000, 300, n_samples)         # Mean £3000, SD £300

# Create ParameterSet
parameters = {
    'effect_standard': effect_standard,
    'effect_new': effect_new,
    'cost_standard': cost_standard,
    'cost_new': cost_new
}

parameter_set = ParameterSet.from_numpy_or_dict(parameters)
print(f"Generated {parameter_set.n_samples} parameter samples")

## Calculate Net Benefits

Now let's calculate the net benefits for our strategies across different WTP thresholds.

In [ ]:
# Calculate net benefits
net_benefit_array = simple_health_economic_model(parameter_set)
print(f"Net benefit array shape: {net_benefit_array.values.shape}")

## EVPI Calculation

Let's calculate the Expected Value of Perfect Information (EVPI).

In [ ]:
# Create DecisionAnalysis object
analysis = DecisionAnalysis(nb_array=net_benefit_array, parameter_samples=parameter_set)

# Calculate EVPI at a specific WTP threshold (e.g., £20,000 per QALY)
wtp_index = 10  # Index for £20,000 in our WTP range
wtp_value = np.linspace(0, 100000, 20)[wtp_index]

# Extract net benefits at this WTP threshold
nb_at_wtp = net_benefit_array.values[:, :, wtp_index]
nb_at_wtp_array = ValueArray.from_numpy(nb_at_wtp, ["Standard Care", "New Treatment"])

# Create new analysis for this WTP
analysis_at_wtp = DecisionAnalysis(nb_array=nb_at_wtp_array, parameter_samples=parameter_set)

# Calculate EVPI
evpi_value = analysis_at_wtp.evpi()
print(f"EVPI at WTP £{wtp_value:,.0f}: £{evpi_value:,.2f}")

## EVPPI Calculation

Let's calculate the Expected Value of Partial Perfect Information (EVPPI) for effectiveness parameters.

In [ ]:
# Calculate EVPPI for effectiveness parameters
evppi_effect = analysis_at_wtp.evppi(parameters_of_interest=["effect_standard", "effect_new"])
print(f"EVPPI for effectiveness parameters: £{evppi_effect:,.2f}")

# Calculate EVPPI for cost parameters
evppi_cost = analysis_at_wtp.evppi(parameters_of_interest=["cost_standard", "cost_new"])
print(f"EVPPI for cost parameters: £{evppi_cost:,.2f}")

## EVSI Calculation

Let's calculate the Expected Value of Sample Information (EVSI) for a hypothetical trial.

In [ ]:
# Define a trial design
standard_arm = DecisionOption(name="Standard Care", sample_size=100)
new_arm = DecisionOption(name="New Treatment", sample_size=100)
trial_design = TrialDesign(arms=[standard_arm, new_arm])

# Define a simple model function for EVSI
def model_func_for_evsi(psa_params):
    return simple_health_economic_model(psa_params)

# Calculate EVSI using the two-loop method
evsi_value_two_loop = evsi(
    model_func=model_func_for_evsi,
    psa_prior=parameter_set,
    trial_design=trial_design,
    method="two_loop",
    n_outer_loops=20,
    n_inner_loops=50
)

print(f"EVSI (two-loop method): £{evsi_value_two_loop:,.2f}")

# Calculate EVSI using the regression method
evsi_value_regression = evsi(
    model_func=model_func_for_evsi,
    psa_prior=parameter_set,
    trial_design=trial_design,
    method="regression",
    n_outer_loops=20
)

print(f"EVSI (regression method): £{evsi_value_regression:,.2f}")

## Plotting Results

Let's create some plots to visualize our results.

In [ ]:
# Plot EVPI vs WTP
wtp_thresholds = np.linspace(0, 100000, 20)
evpi_values = []

for i in range(len(wtp_thresholds)):
    nb_at_wtp = net_benefit_array.values[:, :, i]
    nb_at_wtp_array = ValueArray.from_numpy(nb_at_wtp, ["Standard Care", "New Treatment"])
    analysis_at_wtp = DecisionAnalysis(nb_array=nb_at_wtp_array, parameter_samples=parameter_set)
    evpi_val = analysis_at_wtp.evpi()
    evpi_values.append(evpi_val)

fig, ax = plt.subplots(figsize=(10, 6))
plot_evpi_vs_wtp(evpi_values, wtp_thresholds, ax=ax)
ax.set_title("EVPI vs Willingness-to-Pay Threshold")
plt.show()

In [ ]:
# Plot CEAC
fig, ax = plt.subplots(figsize=(10, 6))
plot_ceac(net_benefit_array, wtp_thresholds, ax=ax)
ax.set_title("Cost-Effectiveness Acceptability Curve")
plt.show()

In [ ]:
# Plot EVSI vs Sample Size
sample_sizes = np.array([50, 100, 200, 500, 1000])
evsi_values = []

for sample_size in sample_sizes:
    trial_design = TrialDesign(arms=[
        DecisionOption(name="Standard Care", sample_size=sample_size//2),
        DecisionOption(name="New Treatment", sample_size=sample_size//2)
    ])
    
    evsi_val = evsi(
        model_func=model_func_for_evsi,
        psa_prior=parameter_set,
        trial_design=trial_design,
        method="two_loop",
        n_outer_loops=10,
        n_inner_loops=20
    )
    evsi_values.append(evsi_val)

fig, ax = plt.subplots(figsize=(10, 6))
plot_evsi_vs_sample_size(evsi_values, sample_sizes, ax=ax)
ax.set_title("EVSI vs Sample Size")
plt.show()

## Summary

This notebook has demonstrated the core functionality of the voiage library:

1. **EVPI Calculation**: We calculated the Expected Value of Perfect Information at different willingness-to-pay thresholds.
2. **EVPPI Calculation**: We calculated the Expected Value of Partial Perfect Information for different parameter groups.
3. **EVSI Calculation**: We calculated the Expected Value of Sample Information using both two-loop and regression methods.
4. **Visualization**: We created plots to visualize the results.

The results show that the voiage library is working correctly and provides valuable insights for health economic decision making.